<a href="https://colab.research.google.com/github/Dongju5/QuantVerse/blob/main/%EC%BD%94%EC%8A%A4%ED%94%BC200_%EB%B2%A0%ED%83%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pykrx

In [ ]:
from pykrx import stock
import pandas as pd

# 최신 코스피 200 지수 데이터를 불러와 월별 수익률을 계산하는 함수
def get_kospi200_monthly_return(start_date, end_date):
    # 일별 데이터를 가져옵니다
    kospi200 = stock.get_index_ohlcv_by_date(start_date, end_date, "1028")  # 코스피 200 지수 코드

    # 날짜 인덱스를 사용하여 매월 첫 번째 날의 종가를 추출합니다
    kospi200 = kospi200.resample('MS').first()  # MS: Month Start

    # 월별 수익률 계산
    kospi200['Monthly Return'] = kospi200['종가'].pct_change()
    kospi200 = kospi200.dropna()  # 결측값 제거
    return kospi200[['종가', 'Monthly Return']]

# 함수 사용 예시: 2020년 10월부터 2024년 11월까지 데이터 불러오기
kospi200_data = get_kospi200_monthly_return("2020-10-01", "2024-11-01")
print(kospi200_data.tail())

코스피 200         종가  Monthly Return
날짜                                
2024-07-01  384.34        0.051603
2024-08-01  380.54       -0.009887
2024-09-01  362.09       -0.048484
2024-10-01  340.72       -0.059018
2024-11-01  337.49       -0.009480


In [ ]:
print(kospi200_data.head())

코스피 200         종가  Monthly Return
날짜                                
2020-11-01  305.95       -0.024114
2020-12-01  352.40        0.151822
2021-01-01  399.88        0.134733
2021-02-01  415.45        0.038937
2021-03-01  414.80       -0.001565


In [ ]:
# Save to CSV
file_path = 'kospi200_monthly_return.csv'
kospi200_data.to_csv(file_path)
print(f"Data saved to {file_path}")

Data saved to kospi200_monthly_return.csv


In [ ]:
!pip install pykrx scikit-learn


개별 주식과 코스피 200 지수의 월별 수익률을 불러오고 베타 계산

In [ ]:
from pykrx import stock
import pandas as pd
from sklearn.linear_model import LinearRegression

# 개별 주식과 코스피 200의 월별 수익률을 불러오는 함수
def get_monthly_returns(ticker, start_date, end_date):
    # 코스피 200 지수 데이터
    kospi200 = stock.get_index_ohlcv_by_date(start_date, end_date, "1028")
    kospi200 = kospi200.resample('MS').first()  # 매월 첫 번째 날의 데이터
    kospi200['KOSPI200 Return'] = kospi200['종가'].pct_change()

    # 개별 주식 데이터
    stock_data = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
    stock_data = stock_data.resample('MS').first()  # 매월 첫 번째 날의 데이터
    stock_data['Stock Return'] = stock_data['종가'].pct_change()

    # 결측값 제거
    kospi200 = kospi200.dropna()
    stock_data = stock_data.dropna()

    # 수익률 데이터를 합치기
    data = pd.DataFrame({
        'KOSPI200': kospi200['KOSPI200 Return'],
        'Stock': stock_data['Stock Return']
    }).dropna()

    return data


In [ ]:
# 베타 계산 함수
def calculate_beta(data):
    X = data['KOSPI200'].values.reshape(-1, 1)
    y = data['Stock'].values.reshape(-1, 1)

    # 회귀 모델 생성 및 훈련
    model = LinearRegression()
    model.fit(X, y)

    # 베타 값 추출
    beta = model.coef_[0][0]
    return beta

# 사용 예시: 삼성전자(005930)와 코스피 200의 월별 수익률 및 베타 계산
ticker = "005930"  # 삼성전자 티커
start_date = "2020-10-01"
end_date = "2024-11-01"

# 월별 수익률 데이터 불러오기
data = get_monthly_returns(ticker, start_date, end_date)
print(data.tail())

# 베타 계산
beta = calculate_beta(data)
print(f"{ticker}의 베타 값: {beta}")

#5개년 베타값 평균 함

            KOSPI200     Stock
날짜                            
2024-07-01  0.051603  0.080581
2024-08-01 -0.009887  0.015892
2024-09-01 -0.048484 -0.104693
2024-10-01 -0.059018 -0.176075
2024-11-01 -0.009480 -0.048940
005930의 베타 값: 1.1806671609575334


In [ ]:
# 월별 수익률 데이터 상위 5개 행 출력
print(data.head())

            KOSPI200     Stock
날짜                            
2020-11-01 -0.024114 -0.022147
2020-12-01  0.151822  0.181185
2021-01-01  0.134733  0.224189
2021-02-01  0.038937  0.000000
2021-03-01 -0.001565  0.007229


In [ ]:
from pykrx import stock

# 코스피 200 종목의 티커 목록 가져오기
kospi200_tickers = stock.get_index_portfolio_deposit_file("1028")  # 코스피 200 지수 코드
print(kospi200_tickers)  # 티커 목록 확인


['005930', '000660', '373220', '207940', '005380', '068270', '000270', '105560', '035420', '055550', '005490', '010130', '012330', '028260', '051910', '032830', '006400', '138040', '012450', '086790', '003670', '000810', '259960', '035720', '329180', '066570', '015760', '033780', '034020', '009540', '011200', '017670', '402340', '267260', '316140', '003550', '024110', '030200', '096770', '034730', '000100', '323410', '018260', '326030', '047050', '042700', '010140', '009150', '086280', '003490', '352820', '042660', '005830', '450080', '090430', '010950', '064350', '047810', '267250', '079550', '180640', '051900', '006800', '034220', '036570', '251270', '011790', '161390', '029780', '021240', '003230', '071050', '454910', '128940', '010120', '005940', '032640', '010620', '016360', '302440', '011170', '271560', '241560', '097950', '078930', '066970', '011070', '022100', '028050', '036460', '011780', '035250', '175330', '272210', '000150', '009830', '006260', '000720', '039490', '004020',

In [ ]:
from pykrx import stock
import pandas as pd
from sklearn.linear_model import LinearRegression
from datetime import datetime

# 개별 주식과 코스피 200의 월별 수익률을 불러오는 함수
def get_monthly_returns(ticker, start_date, end_date):
    try:
        # 코스피 200 지수 데이터
        kospi200 = stock.get_index_ohlcv_by_date(start_date, end_date, "1028")
        kospi200 = kospi200.resample('MS').first()  # 매월 첫 번째 날의 데이터
        kospi200['KOSPI200 Return'] = kospi200['종가'].pct_change()

        # 개별 주식 데이터
        stock_data = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
        stock_data = stock_data.resample('MS').first()  # 매월 첫 번째 날의 데이터
        stock_data['Stock Return'] = stock_data['종가'].pct_change()

        # 결측값 제거
        kospi200 = kospi200.dropna()
        stock_data = stock_data.dropna()

        # 수익률 데이터를 합치기
        data = pd.DataFrame({
            'KOSPI200': kospi200['KOSPI200 Return'],
            'Stock': stock_data['Stock Return']
        }).dropna()

        return data
    except Exception as e:
        print(f"{ticker} 데이터 로드 중 오류 발생: {e}")
        return None

# 베타 계산 함수
def calculate_beta(data):
    if data is None or data.empty:
        return None

    X = data['KOSPI200'].values.reshape(-1, 1)
    y = data['Stock'].values.reshape(-1, 1)

    # 회귀 모델 생성 및 훈련
    model = LinearRegression()
    model.fit(X, y)

    # 베타 값 추출
    beta = model.coef_[0][0]
    return beta

# 모든 코스피 200 종목에 대해 베타 계산 및 업데이트
kospi200_tickers = stock.get_index_portfolio_deposit_file("1028")  # 코스피 200 지수 코드
start_date = "2020-10-01"
end_date = datetime.today().strftime('%Y-%m-%d')  # 오늘 날짜를 종료 날짜로 설정

# 빈 데이터프레임 생성
beta_values = pd.DataFrame(columns=['Ticker', 'Beta'])

# 루프를 통해 각 티커에 대해 베타 계산
for ticker in kospi200_tickers:
    data = get_monthly_returns(ticker, start_date, end_date)
    if data is not None:
        beta = calculate_beta(data)
        if beta is not None:
            # pd.concat을 사용해 새로운 행을 추가
            new_row = pd.DataFrame({'Ticker': [ticker], 'Beta': [beta]})
            beta_values = pd.concat([beta_values, new_row], ignore_index=True)
            print(f"{ticker}의 베타 값: {beta}")
        else:
            print(f"{ticker}의 베타 계산 불가: 데이터 부족")
    else:
        print(f"{ticker}의 데이터 불러오기 실패")

print(beta_values)




<ipython-input-12-c552e3380ff9>:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  beta_values = pd.concat([beta_values, new_row], ignore_index=True)


005930의 베타 값: 1.1806671609575334
000660의 베타 값: 1.4490941054945132
373220의 베타 값: 1.2391109359926549
207940의 베타 값: 0.5338979453273003
005380의 베타 값: 0.7958440007886163
068270의 베타 값: 1.1060295264137743
000270의 베타 값: 0.8345108723243051
105560의 베타 값: 0.5095962484663699
055550의 베타 값: 0.42973807567163175
035420의 베타 값: 0.9651578062112957
005490의 베타 값: 1.0742285132056857
010130의 베타 값: 0.13404955975641594
012330의 베타 값: 0.8841900598875948
028260의 베타 값: 0.6263482303164191
051910의 베타 값: 1.067118363416714
032830의 베타 값: 0.3870272285311783
006400의 베타 값: 1.3259115314038858
138040의 베타 값: 0.7381002197050464
012450의 베타 값: 0.9681899348869674
086790의 베타 값: 0.46426367450015427
003670의 베타 값: 1.9797263613116214
000810의 베타 값: 0.0845044157969389
259960의 베타 값: 1.236994305036556
035720의 베타 값: 1.4859915096723275
329180의 베타 값: 0.27269130789013774
066570의 베타 값: 1.3418994271827225
015760의 베타 값: 0.5664990308603256
033780의 베타 값: 0.07255705479792837
034020의 베타 값: 0.7633459570901552
009540의 베타 값: 1.0261226320674866
011200의

In [ ]:
# 결과를 엑셀 파일로 저장
beta_values.to_excel("kospi200_beta_values.xlsx", index=False)
print("베타 값이 kospi200_beta_values.xlsx 파일로 저장되었습니다.")


베타 값이 kospi200_beta_values.xlsx 파일로 저장되었습니다.
